In [1]:
import numpy as np
from functools import reduce

In [25]:
HALF_PI = np.pi / 2
DEG_TO_RAD = np.pi / 180
sin = np.sin
cos = np.cos
arcsin = np.arcsin
arccos = np.arccos
arctan = np.arctan2
norm = np.linalg.norm
#list of parameters per joint [r, d, alpha]
NUM_JOINTS = 6
PRECISION = 7

PARAMETERS_UR3e = [[0,       .15185, HALF_PI],
              [-.24355, 0,      0],
              [-.2132,  0,      0],
              [0,       .13105, HALF_PI],
              [0,       .08535, -HALF_PI],
              [0,       .0921,  0],
							[-.0535, 	.059, 	0]]

PARAMETERS_UR3 = [[0,       .1519, HALF_PI],
              [-.24365, 0,      0],
              [-.21325,  0,      0],
              [0,       .11235, HALF_PI],
              [0,       .08535, -HALF_PI],
              [0,       .0819,  0],
							[-.0535, 	.059, 	0]]

LL = [0.152, 0.120, 0.244, 0.093, 0.213, 0.083, 0.083, 0.082, 0.0535, 0.052]
BASE_COORD = np.array([-0.15, 0.15, 0.01])

ANGLE_OFFSET = np.array([HALF_PI * 2, 0, 0, -HALF_PI, 0, 0, 0], dtype=float)

trans_params = lambda theta, r, d, alpha: np.array(
  [[cos(theta), -sin(theta) * cos(alpha), sin(theta) * sin(alpha),  r * cos(theta)],
   [sin(theta), cos(theta) * cos(alpha),  -cos(theta) * sin(alpha), r * sin(theta)],
   [0,          sin(alpha),               cos(alpha),               d],
   [0,          0,                        0,                        1]])
trans_theta_joint = lambda theta, joint_index: trans_params(
  theta * DEG_TO_RAD, 
  *(PARAMETERS_UR3[joint_index]))

def lab_fk(theta1, theta2, theta3, theta4, theta5, theta6):
	# Initialize the angles 
	print("Forward kinematics calculated:\n")

	theta = np.array([theta1,theta2,theta3,theta4,theta5,theta6,0], dtype=float)
	T = np.eye(4)

	##### Your Code Starts Here #####
	# Fill in scripts from lab3 here
	for i in range(NUM_JOINTS):
		T = np.matmul(T, trans_theta_joint(theta[i], i))

	##### Your Code Ends Here #####
	print(str(T) + "\n")

	theta = theta + ANGLE_OFFSET
	# angles[0] = theta1 + np.pi
	# angles[1] = theta2
	# angles[2] = theta3
	# angles[3] = theta4 - (0.5*np.pi)
	# angles[4] = theta5
	# angles[5] = theta6
	return theta[:6]

In [26]:
def inverse_kinematics(xWgrip, yWgrip, zWgrip, yaw_WgripDegree):
	angles = np.array([0, 0, 0, 0, 0, 0], dtype=float)
	##### Your Code Starts Here #####
	# TODO: Function that calculates an elbow up 
	# inverse kinematics solution for the UR3

	# Step 1: find gripper position relative to the base of UR3,
	# and set theta_5 equal to -pi/2
	grip = np.array([xWgrip, yWgrip, zWgrip]) - BASE_COORD
	angles[4] = -HALF_PI
	print(angles[4])

	# Step 2: find x_cen, y_cen, z_cen
	yaw = yaw_WgripDegree * DEG_TO_RAD
	wrist_center = grip - np.array([LL[8] * cos(yaw), 
													LL[8] * sin(yaw), 
													0]) 

	# Step 3: find theta_1
	x, y = wrist_center[0], wrist_center[1]
	d = np.sqrt(x*x + y*y)
	
	rectL = LL[1] - LL[3] + LL[5]
	angles[0] = HALF_PI - arctan(x, y) - arcsin(rectL / d)
	#HALF_PI - arctan(y, x) - arcsin(rectL / d)

	# Step 4: find theta_6 
		# based on theta 1 and yaw angle, equals 0 when 9 parallel to 4 and 6
	angles[5] = HALF_PI + angles[0] - yaw

	# Step 5: find x3_end, y3_end, z3_end
	x3 = wrist_center - [LL[6]*cos(angles[0]) - rectL * sin(angles[0]),
											LL[6]*sin(angles[0]) + rectL * cos(angles[0]),
																							-LL[9] - LL[7]]

	# Step 6: find theta_2, theta_3, theta_4
	L1, L3, L5 = LL[0], LL[2], LL[4]
	z = x3[2] - L1
	l = norm(x3 - [0,0,L1])
	angles[2] = HALF_PI*2 - arccos((L3*L3+L5*L5-l*l) / (2*L3*L5))
	angles[1] = -(arcsin(L5*sin(angles[2]) / l) + arcsin(z / l))
	angles[3] = -angles[1] - angles[2]

	##### Your Code Ends Here #####

	# obtain angles from forward kinematics function
	#angles = lab_fk(*angles)

	return angles

In [27]:
def solution(x,y,z,t):
  #define length parameters
  L1 = 0.152
  L2 = 0.120
  L3 = 0.244
  L4 = 0.093
  L5 = 0.213
  L6 = 0.083
  L7 = 0.083
  L8 = 0.082
  L9 = 0.0535
  L10= 0.059

  Zcen = z - 0.01
  Xcen = x-L9*np.cos(t/180*np.pi)+0.15
  Ycen = y-L9*np.sin(t/180*np.pi)-0.15
  
  theta1 = 90. - np.arctan2(Xcen,Ycen)/np.pi*180 - np.arcsin((L6+0.027)/(Xcen**2+Ycen**2)**0.5)/np.pi*180
  y3end = Ycen - np.sin(theta1/180*np.pi)*L7 - np.cos(theta1/180*np.pi)*(L6+0.027)
  x3end = Xcen - np.cos(theta1/180*np.pi)*L7 + np.sin(theta1/180*np.pi)*(L6+0.027)
  z3end = Zcen + L10 + L8
    
  theta6 =  90. - t + theta1
  theta5 = -90.
  L = (x3end**2+y3end**2+(z3end-L1)**2)**0.5
  theta3 = 180 - np.arccos((L3**2+L5**2-L**2)/2/L3/L5)*180/np.pi
  theta2 = -np.arcsin((z3end-L1)/L)/np.pi*180 - np.arcsin(np.sin(np.pi-theta3*np.pi/180)*L5/L)/np.pi*180
  theta4 = - theta3 - theta2
  return [theta1,theta2,theta3,theta4,theta5,theta6]

In [29]:
print(inverse_kinematics(0.2,0.4,0.05,45)/DEG_TO_RAD)
print(solution(0.2,0.4,0.05,45))

-1.5707963267948966
[ 17.25873718 -52.01841823 105.11397047 -53.09555224 -90.
  62.25873718]
[17.258737179784482, -53.36533049227521, 104.91023226949032, -51.54490177721511, -90.0, 62.25873717978448]


In [19]:
a = np.array([0, 0, 0, 0, 0, 0], dtype=float)
a[0] = HALF_PI
a

array([1.57079633, 0.        , 0.        , 0.        , 0.        ,
       0.        ])